In [1]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Pre-processing: normalizing
def normalize(X):
    # max scale for images 255= 2**8= 8 bit grayscale for each channel
    return (X - X.mean(axis=0)) #/ X.std(axis=0)

X_train, X_val, X_test = normalize(X=X_train), normalize(X=X_val), normalize(X=X_test)

In [3]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        self.L = L # layers
        self.C = C # classes
        self.losses = {'train':[], 'train_acc':[], 
                       'valid':[], 'valid_acc':[], 
                       'test':[], 'test_acc':[]}
        
        self.model = []
        self.W_fixed = []
        self.grads = []
        self.dy_prev = np.zeros((1, C))
        self.y_prev = np.zeros((1, C))
        low, high = -1, 1
        
        # Input layer: weights/ biases
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.), b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)
        # Input layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})

        # Hidden layers: weights/ biases
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        # Hidden layer: gradients
        grad_L = []
        for _ in range(L):
            grad_L.append({key: np.zeros_like(val) for key, val in self.model[1][0].items()})
        self.grads.append(grad_L)
        
        # Output layer: weights/ biases
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.), b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        # Output layer: gradients
        self.grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        
#         dX = dout @ W.T # vanilla Backprop
        dX = dout @ W_fixed.T # fba backprop

        return dX, dW, db

    def train_forward(self, X, train):
        caches, ys = [], []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, nl_cache = l.tanh_forward(X=y)
#         y, nl_cache = l.sigmoid_forward(X=y) # non-linearity/ activation
#         y -= l.sigmoid(0.0) # zero-centered/ mean
#         y *= 2.0 # uni-var/ std
        if train:
            caches.append((fc_cache, nl_cache))
        X = y.copy() # pass to the next layer
        
        # Hidden layers
        fc_caches, nl_caches = [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, nl_cache = l.tanh_forward(X=y)
#             y, nl_cache = l.sigmoid_forward(X=y) # non-linearity/ activation
#             y -= l.sigmoid(0.0) # zero-centered/ mean
#             y *= 2.0 # uni-var/ std
            X = y.copy() # pass to next layer
            if train:
                fc_caches.append(fc_cache)
                nl_caches.append(nl_cache)
        if train:
            caches.append((fc_caches, nl_caches)) # caches[1]            
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        y_prob = l.softmax(X=y)
        if train:
            caches.append(fc_cache)

        return y_prob, caches # for backpropating the error

    def cross_entropy(self, y_prob, y_train):
        m = y_prob.shape[0]

        #         prob = l.softmax(y_pred)
        log_like = -np.log(y_prob[range(m), y_train] + l.eps) # to avoid the devision by zero
        data_loss = np.sum(log_like) / m

        return data_loss

    def dcross_entropy(self, y_prob, y_train): # this is equal for both since the reg_loss (noise) derivative is ZERO.
        m = y_prob.shape[0]

        #         grad_y = l.softmax(y_pred)
        grad_y = y_prob
        grad_y[range(m), y_train] -= 1.
        grad_y /= m

        return grad_y

    def loss_function(self, y_prob, y_train):
        
        loss = self.cross_entropy(y_prob, y_train) # softmax is included
        dy = self.dcross_entropy(y_prob, y_train) # dsoftmax is included

        return loss, dy
        
    def train_backward(self, dy, caches, y):
        grads = self.grads.copy() # initialized by Zero in every iteration/epoch
#         dy_prev = self.dy_prev.copy() # for temporal differencing
#         self.dy_prev = dy.copy() # next iteration/ epoch
        y_prev = self.y_prev.copy() # for temporal differencing
        self.y_prev = y.copy() # next iteration/ epoch
        
        # Output layer
        fc_cache = caches[2]
        # softmax_backward is included in dcross_entropy.
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy()
# #         dy =  dy @ self.W_fixed[2].T # done
#         dy_prev =  dy_prev @ self.W_fixed[2].T
        y =  y @ self.W_fixed[2].T # done
        y_prev =  y_prev @ self.W_fixed[2].T
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
        fc_caches, nl_caches = caches[1]
        for layer in reversed(range(self.L)):
#             dy = l.tanh_backward(cache=nl_caches[layer], dout=dy) # diffable function
#             dy = l.sigmoid_backward(cache=nl_caches[layer], dout=dy) # diffable function
#             dy *= dy - dy_prev # temporal diff instead of differentiable function
            dy *= y - y_prev # temporal diff instead of differentiable function
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
# #             dy =  dy @ self.W_fixed[2].T # done
#             dy_prev =  dy_prev @ self.W_fixed[1][layer].T
            y =  y @ self.W_fixed[1][layer].T # done
            y_prev =  y_prev @ self.W_fixed[1][layer].T
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
        fc_cache, nl_cache = caches[0]
#         dy = l.tanh_backward(cache=nl_cache, dout=dy) # diffable function
#         dy = l.sigmoid_backward(cache=nl_caches[layer], dout=dy) # diffable function
#         dy *= dy - dy_prev # temporal diff instead of differentiable function
        dy *= y - y_prev # temporal diff instead of differentiable function
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        y_prob, _ = self.train_forward(X, train=False)
        
        # if self.mode == 'classification':
        y_pred = np.argmax(y_prob, axis=1) # for loss ==err
        
        return y_pred, y_prob
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def sgd(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            y_prob, caches = self.train_forward(X_mini, train=True)
            _, dy = self.loss_function(y_prob, y_mini)
            _, grads = self.train_backward(dy, caches, y_prob)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]

            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
            
            # Training accuracy
            y_pred, y_prob = self.test(X_mini)
            loss, _ = self.loss_function(y_prob, y_mini) # softmax is included in entropy loss function
            self.losses['train'].append(loss)
            acc = np.mean(y_pred == y_mini) # confusion matrix
            self.losses['train_acc'].append(acc)

            # Validate the updated model
            y_pred, y_prob = self.test(X_val)
            valid_loss, _ = self.loss_function(y_prob, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val) # confusion matrix
            self.losses['valid_acc'].append(valid_acc)
            
            # Test the final model
            y_pred, y_prob = nn.test(X_test)
            test_loss, _ = self.loss_function(y_prob, y_test) # softmax is included in entropy loss function
            self.losses['test'].append(test_loss)
            test_acc = np.mean(y_pred == y_test)
            self.losses['test_acc'].append(test_acc)
#             print('Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.
#             format(acc.mean(), acc.std(), loss))
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{}, train loss-{:.4f}, acc-{:.4f}, valid loss-{:.4f}, acc-{:.4f}, test loss-{:.4f}, acc-{:.4f}'.format(
                   iter, loss, acc, valid_loss, valid_acc, test_loss, test_acc))

In [ ]:
# Hyper-parameters
n_iter = 100000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 50 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 10 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 2 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.sgd(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-10, train loss-2.2982, acc-0.1200, valid loss-2.2982, acc-0.1064, test loss-2.3020, acc-0.1102
Iter-20, train loss-2.2774, acc-0.1400, valid loss-2.2981, acc-0.1066, test loss-2.3019, acc-0.1100
Iter-30, train loss-2.2796, acc-0.1000, valid loss-2.2981, acc-0.1062, test loss-2.3018, acc-0.1100
Iter-40, train loss-2.3075, acc-0.0800, valid loss-2.2980, acc-0.1060, test loss-2.3017, acc-0.1103
Iter-50, train loss-2.2879, acc-0.1400, valid loss-2.2979, acc-0.1064, test loss-2.3016, acc-0.1101
Iter-60, train loss-2.3075, acc-0.1800, valid loss-2.2978, acc-0.1064, test loss-2.3015, acc-0.1103
Iter-70, train loss-2.2745, acc-0.1800, valid loss-2.2977, acc-0.1064, test loss-2.3014, acc-0.1108
Iter-80, train loss-2.3075, acc-0.1000, valid loss-2.2976, acc-0.1062, test loss-2.3013, acc-0.1109
Iter-90, train loss-2.3128, acc-0.1200, valid loss-2.2975, acc-0.1062, test loss-2.3012, acc-0.1108
Iter-100, train loss-2.3038, acc-0.1200, valid loss-2.2974, acc-0.1062, test loss-2.3011, acc-0.1111

Iter-830, train loss-2.2956, acc-0.1000, valid loss-2.2910, acc-0.1164, test loss-2.2941, acc-0.1204
Iter-840, train loss-2.2942, acc-0.1000, valid loss-2.2910, acc-0.1172, test loss-2.2940, acc-0.1207
Iter-850, train loss-2.2835, acc-0.1400, valid loss-2.2909, acc-0.1170, test loss-2.2939, acc-0.1206
Iter-860, train loss-2.2704, acc-0.1600, valid loss-2.2908, acc-0.1168, test loss-2.2938, acc-0.1206
Iter-870, train loss-2.2868, acc-0.2000, valid loss-2.2907, acc-0.1172, test loss-2.2937, acc-0.1208
Iter-880, train loss-2.3125, acc-0.1000, valid loss-2.2906, acc-0.1174, test loss-2.2937, acc-0.1211
Iter-890, train loss-2.2918, acc-0.1200, valid loss-2.2905, acc-0.1180, test loss-2.2936, acc-0.1211
Iter-900, train loss-2.3059, acc-0.1200, valid loss-2.2904, acc-0.1176, test loss-2.2935, acc-0.1212
Iter-910, train loss-2.2753, acc-0.2000, valid loss-2.2904, acc-0.1176, test loss-2.2934, acc-0.1212
Iter-920, train loss-2.3018, acc-0.0800, valid loss-2.2903, acc-0.1176, test loss-2.2933, a

Iter-1640, train loss-2.3068, acc-0.1400, valid loss-2.2841, acc-0.1280, test loss-2.2865, acc-0.1303
Iter-1650, train loss-2.2668, acc-0.2200, valid loss-2.2840, acc-0.1278, test loss-2.2864, acc-0.1307
Iter-1660, train loss-2.2876, acc-0.1400, valid loss-2.2839, acc-0.1280, test loss-2.2863, acc-0.1310
Iter-1670, train loss-2.3023, acc-0.1800, valid loss-2.2838, acc-0.1282, test loss-2.2862, acc-0.1310
Iter-1680, train loss-2.2896, acc-0.1400, valid loss-2.2837, acc-0.1280, test loss-2.2861, acc-0.1311
Iter-1690, train loss-2.2816, acc-0.1200, valid loss-2.2837, acc-0.1288, test loss-2.2860, acc-0.1310
Iter-1700, train loss-2.2822, acc-0.1800, valid loss-2.2836, acc-0.1288, test loss-2.2859, acc-0.1310
Iter-1710, train loss-2.2700, acc-0.1400, valid loss-2.2835, acc-0.1288, test loss-2.2858, acc-0.1310
Iter-1720, train loss-2.2979, acc-0.1000, valid loss-2.2834, acc-0.1288, test loss-2.2857, acc-0.1311
Iter-1730, train loss-2.2682, acc-0.1800, valid loss-2.2833, acc-0.1292, test loss

Iter-2450, train loss-2.2910, acc-0.1200, valid loss-2.2772, acc-0.1410, test loss-2.2789, acc-0.1420
Iter-2460, train loss-2.2829, acc-0.0800, valid loss-2.2771, acc-0.1412, test loss-2.2788, acc-0.1421
Iter-2470, train loss-2.2730, acc-0.1600, valid loss-2.2770, acc-0.1412, test loss-2.2787, acc-0.1418
Iter-2480, train loss-2.2745, acc-0.1000, valid loss-2.2769, acc-0.1414, test loss-2.2786, acc-0.1426
Iter-2490, train loss-2.2887, acc-0.0800, valid loss-2.2768, acc-0.1416, test loss-2.2785, acc-0.1427
Iter-2500, train loss-2.2551, acc-0.1000, valid loss-2.2768, acc-0.1416, test loss-2.2784, acc-0.1428
Iter-2510, train loss-2.3081, acc-0.0800, valid loss-2.2767, acc-0.1418, test loss-2.2784, acc-0.1428
Iter-2520, train loss-2.2792, acc-0.0400, valid loss-2.2766, acc-0.1418, test loss-2.2783, acc-0.1430
Iter-2530, train loss-2.2728, acc-0.1200, valid loss-2.2765, acc-0.1420, test loss-2.2782, acc-0.1431
Iter-2540, train loss-2.2653, acc-0.2400, valid loss-2.2764, acc-0.1420, test loss

Iter-3260, train loss-2.2734, acc-0.1800, valid loss-2.2704, acc-0.1526, test loss-2.2715, acc-0.1553
Iter-3270, train loss-2.2961, acc-0.1600, valid loss-2.2703, acc-0.1534, test loss-2.2714, acc-0.1554
Iter-3280, train loss-2.2751, acc-0.1400, valid loss-2.2703, acc-0.1530, test loss-2.2713, acc-0.1556
Iter-3290, train loss-2.2842, acc-0.1800, valid loss-2.2702, acc-0.1536, test loss-2.2712, acc-0.1561
Iter-3300, train loss-2.2849, acc-0.2000, valid loss-2.2701, acc-0.1538, test loss-2.2712, acc-0.1564
Iter-3310, train loss-2.2844, acc-0.2000, valid loss-2.2700, acc-0.1542, test loss-2.2711, acc-0.1565
Iter-3320, train loss-2.2678, acc-0.1600, valid loss-2.2699, acc-0.1550, test loss-2.2710, acc-0.1568
Iter-3330, train loss-2.2691, acc-0.2200, valid loss-2.2698, acc-0.1550, test loss-2.2709, acc-0.1569
Iter-3340, train loss-2.2508, acc-0.1400, valid loss-2.2698, acc-0.1554, test loss-2.2708, acc-0.1570
Iter-3350, train loss-2.2827, acc-0.1800, valid loss-2.2697, acc-0.1554, test loss

...

Iter-33740, train loss-2.1122, acc-0.3600, valid loss-2.0745, acc-0.4456, test loss-2.0558, acc-0.4560
Iter-33750, train loss-2.0670, acc-0.4000, valid loss-2.0744, acc-0.4454, test loss-2.0557, acc-0.4562
Iter-33760, train loss-2.0817, acc-0.4200, valid loss-2.0744, acc-0.4452, test loss-2.0557, acc-0.4562
Iter-33770, train loss-2.0930, acc-0.4200, valid loss-2.0744, acc-0.4456, test loss-2.0556, acc-0.4561
Iter-33780, train loss-2.0404, acc-0.3200, valid loss-2.0743, acc-0.4456, test loss-2.0556, acc-0.4562
Iter-33790, train loss-2.0696, acc-0.3800, valid loss-2.0743, acc-0.4454, test loss-2.0555, acc-0.4564
Iter-33800, train loss-2.0647, acc-0.4000, valid loss-2.0742, acc-0.4454, test loss-2.0555, acc-0.4564
Iter-33810, train loss-2.0601, acc-0.4000, valid loss-2.0741, acc-0.4456, test loss-2.0554, acc-0.4565
Iter-33820, train loss-2.0189, acc-0.4600, valid loss-2.0741, acc-0.4452, test loss-2.0554, acc-0.4565
Iter-33830, train loss-2.0805, acc-0.4200, valid loss-2.0741, acc-0.4452,

Iter-34540, train loss-2.0302, acc-0.5600, valid loss-2.0705, acc-0.4476, test loss-2.0514, acc-0.4589
Iter-34550, train loss-2.0226, acc-0.5000, valid loss-2.0705, acc-0.4474, test loss-2.0513, acc-0.4589
Iter-34560, train loss-2.0381, acc-0.4600, valid loss-2.0704, acc-0.4474, test loss-2.0513, acc-0.4588
Iter-34570, train loss-2.0612, acc-0.5000, valid loss-2.0704, acc-0.4474, test loss-2.0512, acc-0.4589
Iter-34580, train loss-2.0377, acc-0.5200, valid loss-2.0703, acc-0.4474, test loss-2.0511, acc-0.4591
Iter-34590, train loss-2.0208, acc-0.4800, valid loss-2.0703, acc-0.4484, test loss-2.0511, acc-0.4592
Iter-34600, train loss-2.0741, acc-0.4200, valid loss-2.0702, acc-0.4474, test loss-2.0510, acc-0.4592
Iter-34610, train loss-2.0747, acc-0.4000, valid loss-2.0702, acc-0.4484, test loss-2.0510, acc-0.4591
Iter-34620, train loss-2.0664, acc-0.4400, valid loss-2.0701, acc-0.4478, test loss-2.0509, acc-0.4591
Iter-34630, train loss-2.0357, acc-0.4600, valid loss-2.0701, acc-0.4476,

Iter-35340, train loss-2.0888, acc-0.2800, valid loss-2.0665, acc-0.4492, test loss-2.0470, acc-0.4603
Iter-35350, train loss-2.1081, acc-0.3800, valid loss-2.0665, acc-0.4488, test loss-2.0469, acc-0.4603
Iter-35360, train loss-2.0638, acc-0.4200, valid loss-2.0664, acc-0.4492, test loss-2.0469, acc-0.4606
Iter-35370, train loss-2.0326, acc-0.4400, valid loss-2.0664, acc-0.4490, test loss-2.0468, acc-0.4604
Iter-35380, train loss-2.0243, acc-0.4800, valid loss-2.0663, acc-0.4490, test loss-2.0468, acc-0.4605
Iter-35390, train loss-2.0534, acc-0.5200, valid loss-2.0663, acc-0.4490, test loss-2.0467, acc-0.4606
Iter-35400, train loss-2.0319, acc-0.5200, valid loss-2.0662, acc-0.4490, test loss-2.0467, acc-0.4605
Iter-35410, train loss-2.0400, acc-0.4600, valid loss-2.0662, acc-0.4488, test loss-2.0466, acc-0.4604
Iter-35420, train loss-2.0931, acc-0.4200, valid loss-2.0661, acc-0.4488, test loss-2.0465, acc-0.4607
Iter-35430, train loss-2.0729, acc-0.4000, valid loss-2.0661, acc-0.4492,

Iter-36140, train loss-2.0379, acc-0.4600, valid loss-2.0626, acc-0.4514, test loss-2.0427, acc-0.4634
Iter-36150, train loss-2.0428, acc-0.4600, valid loss-2.0626, acc-0.4514, test loss-2.0426, acc-0.4634
Iter-36160, train loss-2.0331, acc-0.5000, valid loss-2.0625, acc-0.4514, test loss-2.0426, acc-0.4634
Iter-36170, train loss-2.0156, acc-0.4000, valid loss-2.0625, acc-0.4508, test loss-2.0425, acc-0.4635
Iter-36180, train loss-2.0435, acc-0.4800, valid loss-2.0624, acc-0.4510, test loss-2.0424, acc-0.4638
Iter-36190, train loss-2.0339, acc-0.5000, valid loss-2.0624, acc-0.4510, test loss-2.0424, acc-0.4638
Iter-36200, train loss-2.0722, acc-0.4200, valid loss-2.0623, acc-0.4514, test loss-2.0423, acc-0.4640
Iter-36210, train loss-2.0981, acc-0.3200, valid loss-2.0623, acc-0.4516, test loss-2.0423, acc-0.4638
Iter-36220, train loss-2.0408, acc-0.4400, valid loss-2.0622, acc-0.4514, test loss-2.0422, acc-0.4638
Iter-36230, train loss-1.9945, acc-0.5800, valid loss-2.0622, acc-0.4516,

Iter-36940, train loss-1.9920, acc-0.5600, valid loss-2.0587, acc-0.4532, test loss-2.0384, acc-0.4663
Iter-36950, train loss-2.0666, acc-0.3800, valid loss-2.0587, acc-0.4528, test loss-2.0383, acc-0.4663
Iter-36960, train loss-2.1252, acc-0.3800, valid loss-2.0587, acc-0.4530, test loss-2.0383, acc-0.4665
Iter-36970, train loss-1.9939, acc-0.5400, valid loss-2.0586, acc-0.4530, test loss-2.0382, acc-0.4668
Iter-36980, train loss-2.0513, acc-0.4800, valid loss-2.0585, acc-0.4530, test loss-2.0381, acc-0.4667
Iter-36990, train loss-2.0902, acc-0.3400, valid loss-2.0585, acc-0.4528, test loss-2.0381, acc-0.4668
Iter-37000, train loss-2.0624, acc-0.5000, valid loss-2.0584, acc-0.4530, test loss-2.0380, acc-0.4668
Iter-37010, train loss-2.0681, acc-0.4200, valid loss-2.0584, acc-0.4530, test loss-2.0380, acc-0.4668
Iter-37020, train loss-2.1349, acc-0.3800, valid loss-2.0584, acc-0.4540, test loss-2.0379, acc-0.4671
Iter-37030, train loss-2.0469, acc-0.4800, valid loss-2.0583, acc-0.4538,

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.plot(nn.losses['test'], label='Test loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['train_acc'], label='Train accuracy')
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.plot(nn.losses['test_acc'], label='Test accuracy')
plt.legend()
plt.show()